<a href="https://colab.research.google.com/github/ryanhao1115/ML-for-Fraud-Detection/blob/main/3_Feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Feature engineering
1. Finalize the fileds needed for model training.
2. Conver fileds into appropriate formats. 

In [2]:
import pandas as pd
import numpy as np

In [3]:
## import dataset 
path = '/content/drive/MyDrive/Colab Notebooks/finalproject/'
file = path + 'sales.csv'
df = pd.read_csv(file,index_col=0)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 248825 entries, 0 to 45763
Data columns (total 20 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   distributor   248825 non-null  object 
 1   sales         248825 non-null  object 
 2   branch        248825 non-null  int64  
 3   inv_type      248825 non-null  object 
 4   invoice_no    248825 non-null  int64  
 5   line_item     248825 non-null  int64  
 6   product_no    248825 non-null  object 
 7   prod_cla      248825 non-null  float64
 8   qty           248825 non-null  int64  
 9   total_amt     248825 non-null  float64
 10  sale_price    248825 non-null  float64
 11  status        248825 non-null  object 
 12  ship_qty      248825 non-null  int64  
 13  cust_type     248825 non-null  object 
 14  return        248825 non-null  object 
 15  discount_app  248825 non-null  object 
 16  list_price    248825 non-null  float64
 17  invoice_date  248825 non-null  object 
 18  ship_

In [ ]:
df.nunique()

distributor        639
sales              209
branch              28
inv_type             3
invoice_no       30721
product_no        1094
prod_cla             7
qty                202
total_amt        16112
sale_price        8598
ship_qty           202
cust_type           20
return               2
discount_app         2
list_price        6720
inv_ship_days      133
dtype: int64

In [ ]:
(df['return'] == 'No').sum()

228413

In [ ]:
df.head()

In [ ]:
df.corr()

,branch,invoice_no,line_item,prod_cla,qty,total_amt,sale_price,ship_qty,list_price
branch,1.000000,-0.049594,-0.121279,-0.122295,-0.172125,-0.085552,-0.119623,-0.179482,-0.120791
invoice_no,-0.049594,1.000000,-0.119274,-0.006354,-0.073161,-0.090843,0.017959,-0.051587,0.018929
line_item,-0.121279,-0.119274,1.000000,0.008218,0.102291,0.099527,0.002805,0.102599,0.002651
prod_cla,-0.122295,-0.006354,0.008218,1.000000,0.018270,-0.001710,0.007406,0.018294,0.007674
qty,-0.172125,-0.073161,0.102291,0.018270,1.000000,0.605014,-0.006894,0.947562,-0.006931
total_amt,-0.085552,-0.090843,0.099527,-0.001710,0.605014,1.000000,-0.397724,0.605055,-0.398421
sale_price,-0.119623,0.017959,0.002805,0.007406,-0.006894,-0.397724,1.000000,-0.006702,0.999939
ship_qty,-0.179482,-0.051587,0.102599,0.018294,0.947562,0.605055,-0.006702,1.000000,-0.006719
list_price,-0.120791,0.018929,0.002651,0.007674,-0.006931,-0.398421,0.999939,-0.006719,1.000000


In [ ]:
df.describe()

,branch,invoice_no,line_item,prod_cla,qty,total_amt,sale_price,ship_qty,list_price
count,248825.000000,2.488250e+05,248825.000000,248825.000000,248825.000000,2.488250e+05,2.488250e+05,248825.000000,2.488250e+05
mean,715065.076918,2.266587e+09,122.754144,11020.197569,1.815529,4.610675e+03,3.198007e+03,1.852169,3.619291e+03
std,56423.284561,5.348107e+08,204.562274,12.469784,11.153265,2.801579e+04,1.438295e+04,10.292674,1.634524e+04
min,72010.000000,1.103721e+09,10.000000,9999.000000,-542.000000,-4.004956e+06,-3.069569e+04,0.000000,-3.560700e+04
25%,720008.000000,2.111335e+09,40.000000,11020.000000,1.000000,3.982300e+02,4.031900e+02,1.000000,4.564300e+02
50%,720013.000000,2.111365e+09,80.000000,11020.000000,1.000000,2.094830e+03,2.123890e+03,1.000000,2.400000e+03
75%,720021.000000,2.111394e+09,130.000000,11020.000000,1.000000,4.017970e+03,3.628320e+03,1.000000,4.100000e+03
max,720037.000000,4.200000e+09,3960.000000,11060.000000,1189.000000,2.051452e+06,4.004956e+06,1189.000000,4.525600e+06


In [ ]:
## Analyze the relationship among 'branch','distributor' and 'sales'
df_b_s = df.groupby(['branch','sales'],as_index=False)['qty'].count()
df_b_s.sales.duplicated().sum()

In [ ]:
df_b_s = df.groupby(['branch','distributor'],as_index=False)['qty'].count()
df_b_s.distributor.duplicated().sum()

175

In [ ]:
df_b_s = df.groupby(['sales','distributor'],as_index=False)['qty'].count()
df_b_s.distributor.duplicated().sum()

801

In [4]:
def count_days(df):
  '''
  convert date fields to date type.  
  Calculate the difference between invoice date and ship date in days
  '''
  df['ship_date'] =  pd.to_datetime(df['ship_date'])
  df['invoice_date'] =  pd.to_datetime(df['invoice_date'])
  df['inv_ship_days'] = (df['ship_date']-df['invoice_date']).dt.days
  return df


In [5]:
df = count_days(df)

In [6]:
df['inv_ship_days'].describe()

count    248825.000000
mean         19.276094
std          52.177451
min          -1.000000
25%           2.000000
50%           3.000000
75%           6.000000
max         239.000000
Name: inv_ship_days, dtype: float64

In [7]:
def remove_unnecessary(df):
  '''
  remove all unnecessary rows and columns
  '''
  df = df[(df['qty'] > 0) & (df['sale_price'] > 0)]
  col = ['line_item','request_ship','status','ship_date','invoice_date']
  df = df.drop(columns=col)
  return df

In [8]:
df = remove_unnecessary(df)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 228601 entries, 0 to 42431
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   distributor    228601 non-null  object 
 1   sales          228601 non-null  object 
 2   branch         228601 non-null  int64  
 3   inv_type       228601 non-null  object 
 4   invoice_no     228601 non-null  int64  
 5   product_no     228601 non-null  object 
 6   prod_cla       228601 non-null  float64
 7   qty            228601 non-null  int64  
 8   total_amt      228601 non-null  float64
 9   sale_price     228601 non-null  float64
 10  ship_qty       228601 non-null  int64  
 11  cust_type      228601 non-null  object 
 12  return         228601 non-null  object 
 13  discount_app   228601 non-null  object 
 14  list_price     228601 non-null  float64
 15  inv_ship_days  228601 non-null  int64  
dtypes: float64(4), int64(5), object(7)
memory usage: 29.6+ MB


In [9]:
## Transform 'return' and 'discount_app' fields
def judge_no(cla):
  if cla == 'No':
    return 0
  else:
    return 1

def feature_transform(df):
  '''
  Transform 'return' and 'discount_app' into binomial value.
  Basically, identify yes or no 'return'/'discount_app'
  '''
  df['return'] = df['return'].apply(judge_no)
  df['discount_app'] = df['discount_app'].apply(judge_no)
  return df


In [10]:
df = feature_transform(df)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 228601 entries, 0 to 42431
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   distributor    228601 non-null  object 
 1   sales          228601 non-null  object 
 2   branch         228601 non-null  int64  
 3   inv_type       228601 non-null  object 
 4   invoice_no     228601 non-null  int64  
 5   product_no     228601 non-null  object 
 6   prod_cla       228601 non-null  float64
 7   qty            228601 non-null  int64  
 8   total_amt      228601 non-null  float64
 9   sale_price     228601 non-null  float64
 10  ship_qty       228601 non-null  int64  
 11  cust_type      228601 non-null  object 
 12  return         228601 non-null  int64  
 13  discount_app   228601 non-null  int64  
 14  list_price     228601 non-null  float64
 15  inv_ship_days  228601 non-null  int64  
dtypes: float64(4), int64(7), object(5)
memory usage: 29.6+ MB


In [12]:
## Extract dataset for next steps
path = '/content/drive/MyDrive/Colab Notebooks/finalproject/'
file_csv = path + 'sales_clean.csv'
df.to_csv(file_csv)